## PCA Humanctivity
---
* 데이터 읽어서 컬럼을 300개 이하로 축소(PCA)시킨 후 RandomForest 모델로 분류
* GridSearchCV로 베스트 모델 찾기
* 최종 결과는 테스트 데이터의 정확도와 confusion matrix

In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import my_utils as my

In [55]:
df_train = pd.read_csv("../data/HumanActivityRecognition/train.csv")
df_train.shape

(7352, 563)

In [56]:
df_train.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,STANDING


In [57]:
x_train = df_train.drop(["Activity"], axis=1)
y_train = df_train["Activity"]

In [58]:
y_train.value_counts()

LAYING                1407
STANDING              1374
SITTING               1286
WALKING               1226
WALKING_UPSTAIRS      1073
WALKING_DOWNSTAIRS     986
Name: Activity, dtype: int64

In [59]:
y_train = y_train.map({"LAYING": 0, "STANDING": 1, "SITTING": 2, "WALKING": 3, "WALKING_UPSTAIRS": 4, "WALKING_DOWNSTAIRS": 5})

In [60]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
x_train = ss.fit_transform(x_train)

In [61]:
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

params = {
    "n_estimators": range(15, 20),
    "max_depth": range(5, 8),
    "min_samples_split": range(7, 10)
    
}

pca = PCA(n_components=250)
x_train = pca.fit_transform(x_train)

clf = RandomForestClassifier(random_state=1)
gs = GridSearchCV(clf, params, cv=3)
gs.fit(x_train, y_train)


GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=1),
             param_grid={'max_depth': range(5, 8),
                         'min_samples_split': range(7, 10),
                         'n_estimators': range(15, 20)})

In [62]:
gs.best_score_

0.7894452077036446

In [63]:
gs = gs.best_estimator_

In [64]:
gs.fit(x_train, y_train)

RandomForestClassifier(max_depth=7, min_samples_split=9, n_estimators=19,
                       random_state=1)

#### 테스트

In [65]:
df_test = pd.read_csv("../data/HumanActivityRecognition/test.csv")
df_test.shape

(2947, 563)

In [66]:
x_test = df_test.drop(["Activity"], axis=1)
y_test = df_test["Activity"]
y_test = y_test.map({"LAYING": 0, "STANDING": 1, "SITTING": 2, "WALKING": 3, "WALKING_UPSTAIRS": 4, "WALKING_DOWNSTAIRS": 5})

In [67]:
x_test = ss.transform(x_test)

In [68]:
# pca = PCA(n_components=250)
x_test = pca.transform(x_test)

In [69]:
gs.score(x_test, y_test)

0.8092975907702749